# Backpropagation from Scratch

This notebook explores backpropagation in detail. We'll derive gradients by hand, implement them, and verify our understanding with numerical checks.

**Goal:** Deep understanding of how gradients flow through a neural network.

**Prerequisites:** [backpropagation.md](../neural-networks/backpropagation.md), [calculus.md](../math-foundations/calculus.md)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

## 1. The Chain Rule

Backprop is just the chain rule applied efficiently. Let's start simple.

In [ ]:
# Simple example: f(x) = (x + 2)^2
# Composed: g(x) = x + 2, f(g) = g^2

def f(x):
    return (x + 2) ** 2

def f_derivative_numerical(x, h=1e-5):
    """Numerical derivative."""
    return (f(x + h) - f(x - h)) / (2 * h)

def f_derivative_analytical(x):
    """Analytical derivative via chain rule.
    
    df/dx = df/dg * dg/dx
          = 2g * 1
          = 2(x + 2)
    """
    return 2 * (x + 2)

# Verify
x = 3.0
print(f"f({x}) = {f(x)}")
print(f"Numerical derivative: {f_derivative_numerical(x):.6f}")
print(f"Analytical derivative: {f_derivative_analytical(x):.6f}")

## 2. Computational Graphs

Neural networks are compositions of simple operations. Let's represent them as graphs.

In [ ]:
# Compute: f(x, y, z) = (x + y) * z
#
# Graph:
#   x ───┐
#        ├──[+]──→ q ──┐
#   y ───┘              ├──[*]──→ f
#   z ─────────────────┘
#
# Forward: q = x + y, then f = q * z
# Backward: df/dq = z, df/dz = q
#           df/dx = df/dq * dq/dx = z * 1 = z
#           df/dy = df/dq * dq/dy = z * 1 = z

def forward_graph(x, y, z):
    """Forward pass through computational graph."""
    q = x + y        # Addition
    f = q * z        # Multiplication
    return f, q      # Return intermediate for backprop

def backward_graph(x, y, z, q):
    """Backward pass: compute gradients."""
    # df/df = 1 (starting point)
    df = 1.0
    
    # f = q * z
    dq = df * z      # df/dq = z
    dz = df * q      # df/dz = q
    
    # q = x + y
    dx = dq * 1      # dq/dx = 1
    dy = dq * 1      # dq/dy = 1
    
    return dx, dy, dz

# Test
x, y, z = 2.0, 3.0, 4.0
f_val, q = forward_graph(x, y, z)
dx, dy, dz = backward_graph(x, y, z, q)

print(f"f = (x + y) * z = ({x} + {y}) * {z} = {f_val}")
print(f"Gradients: dx={dx}, dy={dy}, dz={dz}")
print(f"\nVerification:")
print(f"  df/dx should be z = {z}")
print(f"  df/dy should be z = {z}")
print(f"  df/dz should be (x+y) = {x+y}")

## 3. Gradients of Common Operations

Let's derive and verify gradients for operations used in neural networks.

In [ ]:
def numerical_gradient(f, x, h=1e-5):
    """Compute gradient numerically for any function."""
    grad = np.zeros_like(x)
    it = np.nditer(x, flags=['multi_index'])
    
    while not it.finished:
        idx = it.multi_index
        old_val = x[idx]
        
        x[idx] = old_val + h
        f_plus = f(x)
        
        x[idx] = old_val - h
        f_minus = f(x)
        
        grad[idx] = (f_plus - f_minus) / (2 * h)
        x[idx] = old_val
        it.iternext()
    
    return grad

In [ ]:
# Matrix multiply: z = X @ W
# Forward: z_ij = sum_k X_ik * W_kj
# Backward: dL/dX_ik = sum_j dL/dz_ij * W_kj = (dL/dz @ W.T)_ik
#           dL/dW_kj = sum_i X_ik * dL/dz_ij = (X.T @ dL/dz)_kj

class MatMul:
    """Matrix multiply with gradients."""
    
    def forward(self, X, W):
        self.X = X
        self.W = W
        return X @ W
    
    def backward(self, dout):
        """dout is dL/dz."""
        dX = dout @ self.W.T
        dW = self.X.T @ dout
        return dX, dW

# Verify
X = np.random.randn(3, 4)
W = np.random.randn(4, 5)

matmul = MatMul()
z = matmul.forward(X, W)

# Pretend upstream gradient is random
dout = np.random.randn(*z.shape)
dX, dW = matmul.backward(dout)

# Numerical check for dW
def loss_wrt_W(W_test):
    return np.sum((X @ W_test) * dout)  # Surrogate loss

dW_numerical = numerical_gradient(loss_wrt_W, W.copy())

print("Matrix Multiply Gradient Check (dW):")
print(f"  Max difference: {np.abs(dW - dW_numerical).max():.2e}")

In [ ]:
# ReLU: y = max(0, x)
# Gradient: dy/dx = 1 if x > 0 else 0

class ReLU:
    def forward(self, x):
        self.x = x
        return np.maximum(0, x)
    
    def backward(self, dout):
        return dout * (self.x > 0)

# Verify
x = np.array([-2, -1, 0, 1, 2], dtype=float)
relu = ReLU()
y = relu.forward(x)
dx = relu.backward(np.ones_like(y))

print("ReLU:")
print(f"  x = {x}")
print(f"  y = ReLU(x) = {y}")
print(f"  dy/dx = {dx}")

In [ ]:
# Sigmoid: y = 1 / (1 + exp(-x))
# Gradient: dy/dx = y * (1 - y)

class Sigmoid:
    def forward(self, x):
        self.y = 1 / (1 + np.exp(-x))
        return self.y
    
    def backward(self, dout):
        return dout * self.y * (1 - self.y)

# Verify
x = np.linspace(-5, 5, 11)
sigmoid = Sigmoid()
y = sigmoid.forward(x)

# Numerical gradient
def sigmoid_fn(x):
    return 1 / (1 + np.exp(-x))

dx_numerical = numerical_gradient(lambda x: sigmoid_fn(x).sum(), x.copy())
dx_analytical = sigmoid.backward(np.ones_like(y))

print("Sigmoid Gradient Check:")
print(f"  Max difference: {np.abs(dx_analytical - dx_numerical).max():.2e}")

## 4. Softmax + Cross-Entropy

This is the most important gradient to understand for classification.

In [ ]:
def softmax(z):
    """Numerically stable softmax."""
    exp_z = np.exp(z - z.max(axis=-1, keepdims=True))
    return exp_z / exp_z.sum(axis=-1, keepdims=True)

def cross_entropy(probs, y):
    """Cross-entropy loss. y is integer labels."""
    n = len(y)
    return -np.log(probs[np.arange(n), y] + 1e-10).mean()

class SoftmaxCrossEntropy:
    """
    Combined softmax + cross-entropy for efficient gradient.
    
    The beautiful result: dL/dz = probs - one_hot(y)
    """
    
    def forward(self, z, y):
        self.probs = softmax(z)
        self.y = y
        return cross_entropy(self.probs, y)
    
    def backward(self):
        n = len(self.y)
        dz = self.probs.copy()
        dz[np.arange(n), self.y] -= 1
        dz /= n
        return dz

In [ ]:
# Derive and verify the gradient
# 
# Loss: L = -log(p_y) where p = softmax(z)
# 
# For the correct class k = y:
#   dL/dz_k = -1/p_y * d(p_y)/dz_k
#           = -1/p_y * p_y(1 - p_y)  [softmax derivative for same index]
#           = -(1 - p_y)
#           = p_y - 1
#
# For incorrect class j != y:
#   dL/dz_j = -1/p_y * d(p_y)/dz_j
#           = -1/p_y * (-p_y * p_j)  [softmax derivative for different index]
#           = p_j
#
# Combined: dL/dz = probs - one_hot(y)

# Test
z = np.array([[1.0, 2.0, 3.0], [3.0, 2.0, 1.0]])
y = np.array([2, 0])  # True classes

sce = SoftmaxCrossEntropy()
loss = sce.forward(z, y)
dz = sce.backward()

print(f"Logits z:\n{z}")
print(f"\nProbabilities (softmax):\n{sce.probs.round(3)}")
print(f"\nTrue labels: {y}")
print(f"Loss: {loss:.4f}")
print(f"\nGradient dL/dz:\n{dz.round(4)}")

# Numerical verification
def loss_fn(z_flat):
    z_test = z_flat.reshape(z.shape)
    return cross_entropy(softmax(z_test), y)

dz_numerical = numerical_gradient(loss_fn, z.flatten().copy()).reshape(z.shape)
print(f"\nNumerical gradient:\n{dz_numerical.round(4)}")
print(f"\nMax difference: {np.abs(dz - dz_numerical).max():.2e}")

## 5. Full Network Backprop

Now let's trace gradients through a complete network.

In [ ]:
class LinearLayer:
    """Linear layer with gradient computation."""
    
    def __init__(self, in_features, out_features):
        self.W = np.random.randn(in_features, out_features) * np.sqrt(2 / in_features)
        self.b = np.zeros(out_features)
        self.dW = None
        self.db = None
    
    def forward(self, X):
        self.X = X
        return X @ self.W + self.b
    
    def backward(self, dout):
        """
        dout: gradient from upstream, shape (batch, out_features)
        Returns: gradient for input X, shape (batch, in_features)
        """
        # Parameter gradients
        self.dW = self.X.T @ dout
        self.db = dout.sum(axis=0)
        
        # Input gradient (to pass backwards)
        dX = dout @ self.W.T
        return dX

In [ ]:
class SimpleNetwork:
    """
    Simple two-layer network with explicit gradient computation.
    
    Architecture: Input → Linear → ReLU → Linear → Softmax → Loss
    """
    
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.linear1 = LinearLayer(input_dim, hidden_dim)
        self.relu = ReLU()
        self.linear2 = LinearLayer(hidden_dim, output_dim)
        self.softmax_ce = SoftmaxCrossEntropy()
    
    def forward(self, X, y):
        """Forward pass, returns loss."""
        # Store shapes for visualization
        self.shapes = []
        
        # Layer 1: Linear
        z1 = self.linear1.forward(X)
        self.shapes.append(('z1', z1.shape))
        
        # ReLU
        a1 = self.relu.forward(z1)
        self.shapes.append(('a1', a1.shape))
        
        # Layer 2: Linear
        z2 = self.linear2.forward(a1)
        self.shapes.append(('z2', z2.shape))
        
        # Softmax + Cross-Entropy Loss
        loss = self.softmax_ce.forward(z2, y)
        
        return loss
    
    def backward(self):
        """Backward pass, compute all gradients."""
        # Start from loss
        # dL/dz2
        dz2 = self.softmax_ce.backward()
        
        # dL/da1 (through linear2)
        da1 = self.linear2.backward(dz2)
        
        # dL/dz1 (through ReLU)
        dz1 = self.relu.backward(da1)
        
        # dL/dX (through linear1)
        dX = self.linear1.backward(dz1)
        
        return dX
    
    def get_params_and_grads(self):
        """Return all parameters and their gradients."""
        return [
            ('W1', self.linear1.W, self.linear1.dW),
            ('b1', self.linear1.b, self.linear1.db),
            ('W2', self.linear2.W, self.linear2.dW),
            ('b2', self.linear2.b, self.linear2.db),
        ]

In [ ]:
# Create network and test data
net = SimpleNetwork(input_dim=4, hidden_dim=8, output_dim=3)
X = np.random.randn(2, 4)  # 2 samples, 4 features
y = np.array([0, 2])       # Labels

# Forward
loss = net.forward(X, y)
print(f"Loss: {loss:.4f}")
print(f"\nForward pass shapes:")
for name, shape in net.shapes:
    print(f"  {name}: {shape}")

# Backward
dX = net.backward()
print(f"\ndX shape: {dX.shape}")

# Check gradients
print(f"\nGradient shapes:")
for name, param, grad in net.get_params_and_grads():
    print(f"  {name}: param {param.shape}, grad {grad.shape}")

In [ ]:
# Verify gradients numerically
def check_gradient(net, X, y, param_name, threshold=1e-5):
    """Check analytical gradient against numerical."""
    
    # Get the parameter
    if param_name == 'W1':
        param = net.linear1.W
    elif param_name == 'b1':
        param = net.linear1.b
    elif param_name == 'W2':
        param = net.linear2.W
    elif param_name == 'b2':
        param = net.linear2.b
    
    # Compute analytical gradient
    net.forward(X, y)
    net.backward()
    
    for name, p, g in net.get_params_and_grads():
        if name == param_name:
            analytical = g.copy()
            break
    
    # Compute numerical gradient
    numerical = np.zeros_like(param)
    h = 1e-5
    
    it = np.nditer(param, flags=['multi_index'])
    while not it.finished:
        idx = it.multi_index
        old_val = param[idx]
        
        param[idx] = old_val + h
        loss_plus = net.forward(X, y)
        
        param[idx] = old_val - h
        loss_minus = net.forward(X, y)
        
        numerical[idx] = (loss_plus - loss_minus) / (2 * h)
        param[idx] = old_val
        it.iternext()
    
    # Compare
    diff = np.abs(analytical - numerical).max()
    status = "✓" if diff < threshold else "✗"
    print(f"{param_name}: max diff = {diff:.2e} {status}")
    return diff < threshold

print("Gradient Check:")
check_gradient(net, X, y, 'W1')
check_gradient(net, X, y, 'b1')
check_gradient(net, X, y, 'W2')
check_gradient(net, X, y, 'b2')

## 6. Visualizing Gradient Flow

Let's trace how gradients flow through the network.

In [ ]:
# Create a deeper network to see gradient flow
class DeepNetwork:
    def __init__(self, layer_sizes):
        self.layers = []
        self.activations = []
        
        for i in range(len(layer_sizes) - 1):
            self.layers.append(LinearLayer(layer_sizes[i], layer_sizes[i+1]))
            if i < len(layer_sizes) - 2:  # No activation on last layer
                self.activations.append(ReLU())
        
        self.softmax_ce = SoftmaxCrossEntropy()
    
    def forward(self, X, y):
        self.z_values = []  # Store pre-activation values
        self.a_values = [X]  # Store post-activation values
        
        h = X
        for i, layer in enumerate(self.layers):
            z = layer.forward(h)
            self.z_values.append(z)
            
            if i < len(self.activations):
                h = self.activations[i].forward(z)
                self.a_values.append(h)
            else:
                h = z
        
        return self.softmax_ce.forward(h, y)
    
    def backward(self):
        self.gradient_norms = []
        
        dout = self.softmax_ce.backward()
        self.gradient_norms.append(np.linalg.norm(dout))
        
        for i in reversed(range(len(self.layers))):
            dout = self.layers[i].backward(dout)
            if i > 0:
                dout = self.activations[i-1].backward(dout)
            self.gradient_norms.append(np.linalg.norm(dout))
        
        self.gradient_norms.reverse()
        return dout

In [ ]:
# Create a 5-layer network
deep_net = DeepNetwork([10, 20, 20, 20, 20, 5])

X = np.random.randn(32, 10)
y = np.random.randint(0, 5, 32)

loss = deep_net.forward(X, y)
deep_net.backward()

# Plot gradient norms through the network
plt.figure(figsize=(10, 4))
plt.bar(range(len(deep_net.gradient_norms)), deep_net.gradient_norms)
plt.xlabel('Layer (0 = input, last = output)')
plt.ylabel('Gradient Norm')
plt.title('Gradient Flow Through Network')
plt.yscale('log')
plt.show()

print("\nGradient norms by layer:")
for i, norm in enumerate(deep_net.gradient_norms):
    print(f"  Layer {i}: {norm:.4f}")

## 7. Vanishing Gradients

Let's demonstrate the vanishing gradient problem with sigmoid activations.

In [ ]:
class DeepNetworkSigmoid:
    """Deep network with sigmoid activations (to show vanishing gradients)."""
    
    def __init__(self, layer_sizes):
        self.layers = []
        self.activations = []
        
        for i in range(len(layer_sizes) - 1):
            self.layers.append(LinearLayer(layer_sizes[i], layer_sizes[i+1]))
            if i < len(layer_sizes) - 2:
                self.activations.append(Sigmoid())  # Using sigmoid!
        
        self.softmax_ce = SoftmaxCrossEntropy()
    
    def forward(self, X, y):
        h = X
        for i, layer in enumerate(self.layers):
            z = layer.forward(h)
            if i < len(self.activations):
                h = self.activations[i].forward(z)
            else:
                h = z
        return self.softmax_ce.forward(h, y)
    
    def backward(self):
        self.gradient_norms = []
        
        dout = self.softmax_ce.backward()
        self.gradient_norms.append(np.linalg.norm(dout))
        
        for i in reversed(range(len(self.layers))):
            dout = self.layers[i].backward(dout)
            if i > 0:
                dout = self.activations[i-1].backward(dout)
            self.gradient_norms.append(np.linalg.norm(dout))
        
        self.gradient_norms.reverse()

# Compare ReLU vs Sigmoid
sizes = [10, 50, 50, 50, 50, 50, 50, 50, 5]  # 8 layers

relu_net = DeepNetwork(sizes)
sigmoid_net = DeepNetworkSigmoid(sizes)

X = np.random.randn(32, 10)
y = np.random.randint(0, 5, 32)

relu_net.forward(X, y)
relu_net.backward()

sigmoid_net.forward(X, y)
sigmoid_net.backward()

plt.figure(figsize=(10, 4))
plt.plot(relu_net.gradient_norms, 'b-o', label='ReLU')
plt.plot(sigmoid_net.gradient_norms, 'r-s', label='Sigmoid')
plt.xlabel('Layer')
plt.ylabel('Gradient Norm')
plt.title('Vanishing Gradients: ReLU vs Sigmoid')
plt.legend()
plt.yscale('log')
plt.grid(True)
plt.show()

print(f"\nReLU gradient ratio (first/last): {relu_net.gradient_norms[0]/relu_net.gradient_norms[-1]:.2e}")
print(f"Sigmoid gradient ratio (first/last): {sigmoid_net.gradient_norms[0]/sigmoid_net.gradient_norms[-1]:.2e}")

## 8. Exercises

1. **Implement Tanh:** Add a Tanh activation class and verify the gradient.

2. **Batch Normalization:** Implement BatchNorm with its gradient.

3. **Dropout:** Add dropout during forward pass (with proper scaling).

4. **Residual Connection:** Implement a residual block and trace gradients.

5. **Different Loss Functions:** Implement MSE loss and its gradient.

In [ ]:
# Exercise 1: Implement Tanh
# tanh(x) = (e^x - e^-x) / (e^x + e^-x)
# d/dx tanh(x) = 1 - tanh(x)^2

class Tanh:
    def forward(self, x):
        self.y = np.tanh(x)
        return self.y
    
    def backward(self, dout):
        # TODO: Implement
        pass

## Summary

Key insights from this notebook:

1. **Backprop = Chain Rule:** It's just calculus applied systematically.

2. **Local Gradients:** Each operation only needs to know its local gradient.

3. **Gradient Flow:** Gradients multiply as they flow backward. Sigmoid squashes gradients (max derivative 0.25), while ReLU preserves them (derivative 1 for positive inputs).

4. **Softmax + CE:** The combined gradient is elegantly simple: `probs - one_hot(y)`.

5. **Numerical Verification:** Always check gradients during implementation.

**Next:** Apply this understanding to build [attention mechanisms](../transformers/attention.md).